In [1]:
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd 
import time
import re
import requests 

In [8]:
def openPage(urlPage):
    try:
        html = urlopen(urlPage)
        bsObj = BeautifulSoup(html,"html.parser",from_encoding="gb18030")
    except Exception as e:
        print('except:', e)
        if e == "HTTP Error 404: Not Found":
            print(urlPage)
        else:
            time.sleep(3)
            bsObj = getLinks(urlPage)
    return bsObj

In [9]:
def getLinks(urlpage):
    bsObj = openPage(urlpage)
    weeklist = bsObj.find("div",{"class":"erji_list1"}).find_all("li")
    for li in weeklist:
        link = li.find("a")['href'][1:]
        if li not in week_dict.values():
            week_dict[li.find("a").get_text()] = "http://www.chinaivdc.cn/cnic/zyzx/lgzb" + li.find("a")['href'][1:]

In [13]:
week_dict = {}
url = "http://www.chinaivdc.cn/cnic/zyzx/lgzb/index"
getLinks(url + ".htm")
for i in range(1,21):
    getLinks(url + "_" + str(i) + ".htm")

In [14]:
for j in range(10,14):
    week_dict.pop("2011 第" + str(j) + "周")
for j in range(5,10):
    week_dict.pop("2011 第0" + str(j) + "周")
week_dict.pop("2011第04周")

'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201605/t20160520_129710.htm'

In [15]:
print(len(week_dict.values()))

410


In [18]:
print(week_dict.values())

dict_values(['http://www.chinaivdc.cn/cnic/zyzx/lgzb/201903/t20190301_199745.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201902/t20190226_199648.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201902/t20190215_199512.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201902/t20190215_199499.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201902/t20190203_199331.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201901/t20190131_199298.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201901/t20190118_198920.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201901/t20190113_198818.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201901/t20190104_198668.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201901/t20190102_198620.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201812/t20181224_198439.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201812/t20181219_198333.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201812/t20181207_198007.htm', 'http://www.chinaivdc.cn/cnic/zyzx/lgzb/201812/t20181207_198005.htm', 'http:/

In [27]:
PSRdata = []
for k,v in week_dict.items():
    try:
        date = k
        
        res = requests.get(v)
        res.encoding = 'utf-8'
        reg = '<.*?>|&nbsp'
        content = re.sub(reg, "", res.text)                #清理内容，去掉所有HTML标签
        reg_float = '\d+\.\d+'                             #匹配小数，/d+:任意0-多位数字；/.：匹配小数点
        
        #南方
        reg_south = '(南方省份流感检测阳性率|南方省份第).*?[，]'              #a.*?b:匹配一长段字符中以a开头以b结尾的子串
        south = re.search(reg_south,content).group()                             
        S_PSR = re.search(reg_float, south).group()
        s_psr = float(S_PSR)
        
        #北方
        reg_north = '(北方省份流感检测阳性率|北方省份第).*?[，]'      
        north = re.search(reg_north,content).group()                          
        N_PSR = re.search(reg_float, north).group()
        n_psr = float(N_PSR)
        
        #主要问题：当数字为0时，语句表述为"北方省份未检测到流感病毒"，即无法获取数字。
        #教训：try excepte语句分小块写，详细到具体的部分
        
        PSRdata.append([date, s_psr, n_psr])
     
    except Exception as e:
        print(v)
        print('except:', e)
    
    time.sleep(1)

http://www.chinaivdc.cn/cnic/zyzx/lgzb/201810/t20181016_195082.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201809/t20180929_194398.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201809/t20180923_194062.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201809/t20180911_193721.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201808/t20180824_189988.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201808/t20180820_189840.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201808/t20180810_189667.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201808/t20180806_189507.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201807/t20180723_

In [30]:
PSRdata_df = pd.DataFrame(PSRdata, columns = ["time","south_PSR","north_PSR"])
writer = pd.ExcelWriter("C:\\D\\HUST\\毕业设计\\infectious disease data\\flu positive specimens rate\\flu positive specimens rate.xlsx")
PSRdata_df.to_excel(writer,"sheet1",index = False)
writer.save()